## Compare SL images  GEMS Rubin

- author : Sylvie Dagoret-Campagne
- creation date : 2025-05-24
- last update : 2025-05-29 : improve showing same xmin,xmax, ymin,ymax

In [ ]:
import astroquery
print(astroquery.__version__)

https://stackoverflow.com/questions/59699193/how-to-obtain-2d-cutout-of-an-image-from-a-skycoord-position

In [ ]:
import matplotlib.pyplot as plt
from astropy.io import fits
from astropy.wcs import WCS
import os
import re
import numpy as np

In [ ]:
# For Angle conversion
from astropy.coordinates import Angle
import astropy.units as u
from astropy.coordinates import SkyCoord
import pandas as pd
import numpy as np
from astropy.nddata import Cutout2D

In [ ]:
from astropy.visualization import ZScaleInterval, AsinhStretch, ImageNormalize
from astropy.visualization import ZScaleInterval, simple_norm, make_lupton_rgb

In [ ]:
from IPython.display import Image, display

In [ ]:
plt.rcParams["figure.figsize"] = (4,3)
plt.rcParams["axes.labelsize"] = 'xx-large'
plt.rcParams['axes.titlesize'] = 'xx-large'
plt.rcParams['xtick.labelsize']= 'xx-large'
plt.rcParams['ytick.labelsize']= 'xx-large'

In [ ]:
def loadgems_fits_image_with_norm(path):
    from astropy.io import fits
    from astropy.wcs import WCS
    with fits.open(path) as hdul:
        data = hdul[0].data
        wcs = WCS(hdul[0].header)
    vmin, vmax = ZScaleInterval().get_limits(data)
    #norm = simple_norm(data, stretch='asinh', min_cut=vmin, max_cut=vmax)
    norm = simple_norm(data, stretch='asinh', vmin=vmin, vmax=vmax)
    return data, wcs, norm


In [ ]:
def loadlsst_fits_image_with_norm(path):
    """
    Load rubin cutout adapted to rubin LSST
    """
    with fits.open(path) as hdul:
        # where is the image
        data = hdul[1].data
        if data is None:
            raise ValueError(f"No image data in {path}")
        # where is the header that contains WCS
        wcs = WCS(hdul[0].header)
        
    vmin, vmax = ZScaleInterval().get_limits(data)
    norm = simple_norm(data, stretch='asinh', vmin=vmin, vmax=vmax)
    return data, wcs, norm

In [ ]:
def make_lupton_image(data_b, data_g, data_r, stretch=0.02, Q=10):
    return make_lupton_rgb(image_r=data_r, image_g=data_g, image_b=data_b,
                           stretch=stretch, Q=Q)

In [ ]:
def compute_color_difference(data_band1, data_band2):
    """
    Calcule la différence de magnitudes entre deux bandes : -2.5 * log10(b1) + 2.5 * log10(b2)
    """
    with np.errstate(divide='ignore', invalid='ignore'):
        mask = (data_band1 > 0) & (data_band2 > 0)
        diff = np.full_like(data_band1, np.nan)
        diff[mask] = -2.5 * np.log10(data_band1[mask]) + 2.5 * np.log10(data_band2[mask])
    return diff

In [ ]:
def compute_common_limits(diff_maps, quantile_clip=0.01):
    all_values = np.concatenate([d[np.isfinite(d)].ravel() for d in diff_maps])
    vmin = np.quantile(all_values, quantile_clip)
    vmax = np.quantile(all_values, 1 - quantile_clip)
    return vmin, vmax

## Configuration

### Target in ECDFS

- path to article : https://arxiv.org/pdf/1104.0931
- visual selection of the tiles : https://archive.stsci.edu/prepds/gems/browser.html
- path file download : https://archive.stsci.edu/pub/hlsp/gems/v_mk1/

In [ ]:
fig_table1 = "input_figs/table1_gemcandidates.png"
fig_table2 = "input_figs/table2_gemcandidates.png"

In [ ]:
Image(url= fig_table1,width=800)

In [ ]:
Image(url= fig_table2,width=500)

In [ ]:
#15422 44 03:32:38.21 –27:56:53.2 
ra1 = "03:32:38.21 hours"
dec1 = "-27:56:53.2 degrees"
tile1 = 44

#34244 94 03:32:06.45 –27:47:28.6 
ra2 = "03:32:06.45 hours"
dec2 = "-27:47:28.6 degrees"
tile2 = 94
# Je ne trouve pas cette tile ==> FindTileForCutoutGEM
tile2 = 32

#40173 35 03:33:19.45 –27:44:50.0 
ra3 = "03:33:19.45 hours"
dec3 = "-27:44:50.0 degrees"
tile3 = 35

#43242 45 03:31:55.35 –27:43:23.5 
ra4 = "03:31:55.35 hours"
dec4 = "-27:43:23.5 degrees"
tile4 = 45

#46446 47 03:31:35.94 –27:41:48.2 
ra5 = "03:31:35.94 hours"
dec5 = "-27:41:48.2 degrees"
tile5 = 47

#12589 03:31:24.89 −27:58:07.0
ra6 = "03:31:24.89 hours"
dec6 = "-27:58:07.0 degrees"
tile6 = 17

#43797 03:31:31.74 −27:43:00.8 
ra7 = "03:31:31.74 hours"
dec7 = "-27:43:00.8 degrees"
tile7 = 47

#28294 03:31:50.54 −27:50:28.4 
ra8 = "03:31:50.54 hours"
dec8 = "-27:50:28.4 degrees"
tile8 = 33

#36857 03:31:53.24 −27:46:18.9
ra9 = "03:31:53.24 hours"
dec9 = "-27:46:18.9 degrees"
tile9 = 38

#36714 03:32:59.78 −27:46:26.4 
ra10 = "03:32:59.78 hours"
dec10 = "-27:46:26.4 degrees"
tile10 = 37


In [ ]:
ra = Angle(ra10)
print(ra.degree)
dec = Angle(dec10)
print(dec.degree)

In [ ]:
lsstcomcam_targets = {}
# high rank
lsstcomcam_targets["ECDFS_G15422"] = {"field_name": "GEMS-15422", "ra": 53.159208333333325, "dec": -27.94811111111111,"tile":tile1}
lsstcomcam_targets["ECDFS_G34244"] = {"field_name": "GEMS-34244", "ra": 53.02687499999999 , "dec": -27.79127777777778,"tile":tile2}
lsstcomcam_targets["ECDFS_G40173"] = {"field_name": "GEMS-40173", "ra": 53.33104166666666 , "dec": -27.747222222222224,"tile":tile3}
lsstcomcam_targets["ECDFS_G43242"] = {"field_name": "GEMS-43242", "ra": 52.980624999999996 , "dec": -27.72319444444444,"tile":tile4}
lsstcomcam_targets["ECDFS_G46446"] = {"field_name": "GEMS-46446", "ra": 52.89975 , "dec": -27.696722222222224,"tile":tile5}

# low rank
lsstcomcam_targets["ECDFS_G12589"] = {"field_name": "GEMS-12589", "ra": 52.85370833333333, "dec": -27.96861111111111,"tile":tile6}
lsstcomcam_targets["ECDFS_G43797"] = {"field_name": "GEMS-43797", "ra": 52.88224999999999, "dec": -27.71688888888889,"tile":tile7}

lsstcomcam_targets["ECDFS_G28294"] = {"field_name": "GEMS-28294", "ra": 52.960583333333325 , "dec": -27.84122222222222,"tile":tile8}
lsstcomcam_targets["ECDFS_G6857"] = {"field_name": "GEMS-6857", "ra": 52.97183333333333 , "dec": -27.771916666666666,"tile":tile9}
lsstcomcam_targets["ECDFS_G36714"] = {"field_name": "GEMS-36714", "ra": 53.249083333333324, "dec": -27.773999999999997,"tile":tile10}


In [ ]:
df = pd.DataFrame(lsstcomcam_targets).T

In [ ]:
# candidates
#key = "ECDFS_G15422"
key = "ECDFS_G34244"
#key = "ECDFS_G40173"
#key= "ECDFS_G43242"
#key= "ECDFS_G46446"

# unknown
#key = "ECDFS_G12589"
#key = "ECDFS_G43797"
#key = "ECDFS_G28294"
#key = "ECDFS_G6857"
#key = "ECDFS_G36714"

the_target = lsstcomcam_targets[key]
target_ra = the_target["ra"]
target_dec = the_target["dec"]
target_name = the_target["field_name"]
tile_num = the_target["tile"]
target_coord = SkyCoord(ra=target_ra, dec=target_dec, unit=(u.deg, u.deg))

target_title = (
    the_target["field_name"] + f" (ra,dec) = ({target_ra:.2f},{target_dec:.2f}) "
)

### Access to remote files

- visual selection of the tiles : https://archive.stsci.edu/prepds/gems/browser.html
- path file download : https://archive.stsci.edu/pub/hlsp/gems/v_mk1/

### Config

- http://archive.stsci.edu/pub/hlsp/gems/v_mk1/h_gems_v35_mk1.fits
- url = f"http://archive.stsci.edu/pub/hlsp/gems/z_mk1/h_gems_z{tile_num}_mk1.fits"

In [ ]:
toppath_gemscutout = "../2025-05-24-GEMS"
toppath_rubincutout = "../2025-05-24-SL-ECDFS-Fit/data"

In [ ]:
! ls ../2025-05-24-GEMS

In [ ]:
ls ../2025-05-24-SL-ECDFS-Fit/data/G43797

### Choose the good index

In [ ]:
all_gems_id_numbers = [15422,34244,40173,43242,46446, 12589,43797,28294,6857,36714]

In [ ]:
idx = 3
print(all_gems_id_numbers[idx])

In [ ]:
sl_number = all_gems_id_numbers[idx]

In [ ]:
#sl_number = all_gems_id_numbers[idx]
#sl_number = int(re.findall("^ECDFS_G(.*)$",key)[0])

key  = f"ECDFS_G{sl_number}"
has_v = key != "ECDFS_G15422"

the_target = lsstcomcam_targets[key]
target_ra = the_target["ra"]
target_dec = the_target["dec"]
target_name = the_target["field_name"]
tile_num = the_target["tile"]
target_coord = SkyCoord(ra=target_ra, dec=target_dec, unit=(u.deg, u.deg))

target_title = (
    the_target["field_name"] + f" (ra,dec) = ({target_ra:.2f},{target_dec:.2f}) "
)

subpath1 = f"G{sl_number}"
path1_lsst = os.path.join(toppath_rubincutout,subpath1)
path2_gems = toppath_gemscutout

# band 1 : rubin (g,r,i,z,y)
bands1 = ["g","i","z"]
band1_lupton = f"lsst RGB : {tuple(bands1)}"
band1_colors = f"lsst color : {bands1[0]} - {bands1[-1]}"

# band 2 : gem-hst (v,z)
if has_v:
    bands2 = ["v","z"]
    band2_colors = f"gems color : {bands2[0]} - {bands2[-1]}"
    band2_lupton = f"gems RGB : {tuple(bands2)}"
else:
    bands2 = ["z"]
    band2_colors = f"gems color : None "

# field of view : 6 arcsec
fov_deg = 6.0/3600.

corner1 = SkyCoord(target_ra - fov_deg / 2, target_dec - fov_deg / 2, unit="deg")
corner2 = SkyCoord(target_ra + fov_deg / 2, target_dec + fov_deg / 2, unit="deg")


In [ ]:
# --- Remplace par tes fichiers FITS ---
all_files1 = []
all_fullfilenames1 = []
for band1 in bands1:
    file1 = f"cutout_wcs_ECDFS_{subpath1}_{band1}.fits"
    fullfilename1 = os.path.join(path1_lsst ,file1)
    all_files1.append(file1)
    all_fullfilenames1.append(fullfilename1) 

all_files2 = []
all_fullfilenames2 = []
for band2 in bands2:
    file2 = f"cutout_gems_ECDFS_{subpath1}_b{band2}.fits"
    fullfilename2 = os.path.join(path2_gems ,file2)
    all_files2.append(file2)
    all_fullfilenames2.append(fullfilename2) 
    

In [ ]:
all_images1 = [ loadlsst_fits_image_with_norm(path) for path in all_fullfilenames1]

In [ ]:
all_images2 = [ loadgems_fits_image_with_norm(path) for path in all_fullfilenames2]

In [ ]:
lsst_lupton = make_lupton_image(data_b = all_images1[0][0],
                                data_g = all_images1[1][0], 
                                data_r = all_images1[2][0])

if has_v:
    gems_lupton = make_lupton_image(data_b = all_images2[0][0],
                                data_g = 0.5*(all_images2[0][0]+all_images2[1][0]), 
                                data_r = all_images2[1][0])
else:
    gems_lupton = all_images2[0][0]
    

#lsst_diff = -2.5*(np.log10(all_images1[0][0]) - np.log10(all_images1[-1][0]))
#gems_diff = -2.5*(np.log10(all_images2[0][0]) - np.log10(all_images2[-1][0]))

lsst_diff = compute_color_difference(all_images1[0][0], all_images1[-1][0])
common_vmin,common_vmax = compute_common_limits(lsst_diff, quantile_clip=0.01)
lsst_norm = simple_norm(lsst_diff, stretch='linear', vmin=common_vmin, vmax=common_vmax)

gems_diff = compute_color_difference(all_images2[0][0], all_images2[-1][0])
common_vmin,common_vmax = compute_common_limits(gems_diff, quantile_clip=0.01)
gems_norm = simple_norm(gems_diff, stretch='linear', vmin=common_vmin, vmax=common_vmax)


wcs1 =  all_images1[0][1]
wcs2 =  all_images2[0][1]

x1_min, y1_min = wcs1.world_to_pixel(corner1)
x1_max, y1_max = wcs1.world_to_pixel(corner2)

x2_min, y2_min = wcs2.world_to_pixel(corner1)
x2_max, y2_max = wcs2.world_to_pixel(corner2)


print(f"(x1_min,x1_max) = ({x1_min},{x1_max})")
print(f"(x2_min,x2_max) = ({x2_min},{x2_max})")

print(f"(y1_min,y1_max) = ({y1_min},{y1_max})")
print(f"(y2_min,y2_max) = ({y2_min},{y2_max})")


In [ ]:
fig = plt.figure(figsize=(14,12))
                 
ax1 = fig.add_subplot(2, 2, 1, projection=wcs1)
ax1.imshow(lsst_lupton ,origin="lower")
ax1.set_title("LSSTComCam " + band1_lupton)

ax2 = fig.add_subplot(2, 2, 2, projection=wcs2)
ax2.imshow(gems_lupton,origin="lower")
ax2.set_title("HST-GEMS " + band2_lupton)

ax3 = fig.add_subplot(2, 2, 3, projection=wcs1)
im3 = ax3.imshow(lsst_diff,origin="lower",cmap="seismic",norm=lsst_norm)
ax3.set_title(band1_colors)
fig.colorbar(im3, ax=ax3, shrink=0.7)

ax4 = fig.add_subplot(2, 2, 4, projection=wcs2)
im4 = ax4.imshow(gems_diff,origin="lower",cmap="seismic",norm=gems_norm)
ax4.set_title(band2_colors)
fig.colorbar(im4, ax=ax4, shrink=0.7)


# Pour ax1
#ax1.set_xlim(min(x1_min, x1_max), max(x1_min, x1_max))
#ax1.set_ylim(min(y1_min, y1_max), max(y1_min, y1_max))
#ax1.invert_xaxis()  # souvent nécessaire avec les images WCS

# Pour ax2
#ax2.set_xlim(min(x2_min, x2_max), max(x2_min, x2_max))
#ax2.set_ylim(min(y2_min, y2_max), max(y2_min, y2_max))
#ax2.invert_xaxis()

plt.suptitle(key,fontweight="bold",fontsize=20)
plt.tight_layout()

plt.show()


In [ ]:
fig = plt.figure(figsize=(14, 15))

# Image RGB LSST
ax1 = fig.add_subplot(2, 2, 1, projection=wcs1)
ax1.imshow(lsst_lupton, origin="lower")
ax1.set_title("LSSTComCam " + band1_lupton)

# Image RGB GEMS
ax2 = fig.add_subplot(2, 2, 2, projection=wcs2)
ax2.imshow(gems_lupton, origin="lower")
ax2.set_title("HST-GEMS " + band2_lupton)

# Diff LSST
ax3 = fig.add_subplot(2, 2, 3, projection=wcs1)
im3 = ax3.imshow(lsst_diff, origin="lower", cmap="seismic", norm=lsst_norm)
ax3.set_title(band1_colors)
cbar3 = fig.colorbar(im3, ax=ax3, orientation="horizontal", pad=0.1)
cbar3.set_label("color (mag)")

# Diff GEMS
ax4 = fig.add_subplot(2, 2, 4, projection=wcs2)
im4 = ax4.imshow(gems_diff, origin="lower", cmap="seismic", norm=gems_norm)
ax4.set_title(band2_colors)
cbar4 = fig.colorbar(im4, ax=ax4, orientation="horizontal", pad=0.1)
cbar4.set_label("color (mag)")

plt.suptitle(key, fontweight="bold", fontsize=20)
#plt.tight_layout(rect=[0, 0, 1, 0.96])  # laisse de la place au titre
plt.tight_layout()
fig.subplots_adjust(hspace=0.01)

plt.show()
